$y = x^3 + 2x^2y -5xy^2 -3y +2$

Step 1. R1CS real numbers to test

In [31]:
import numpy as np
import random
x = 5#random.randint(1,10)
y = 10#random.randint(1,15)

v1 = x * x
v2 = v1 * x
v3 = y * y
v4 = v2 * y
out = v2 + 2 * v4 - 5 * x * v3 - 3 * y + 2
print(1,out,x,y,v1,v2,v3,v4)

L = np.array([
    [0,0,1,0,0,0,0,0],
    [0,0,0,0,1,0,0,0],
    [0,0,0,1,0,0,0,0],
    [0,0,0,0,0,1,0,0],
    [0,0,-5,0,0,0,0,0]
    ])

R = np.array([
    [0,0,1,0,0,0,0,0],
    [0,0,1,0,0,0,0,0],
    [0,0,0,1,0,0,0,0],
    [0,0,0,1,0,0,0,0],
    [0,0,0,0,0,0,1,0]
    ])

O = np.array([
    [0,0,0,0,1,0,0,0],
    [0,0,0,0,0,1,0,0],
    [0,0,0,0,0,0,1,0],
    [0,0,0,0,0,0,0,1],
    [-2,1,0,3,0,-1,0,-2]
    ])

w = np.array([1,out,x,y,v1,v2,v3,v4])

# assert all(np.multiply(np.dot(L,w), np.dot(R,w)) == np.dot(O,w)), "result contains an inequality"
result = O.dot(w) == np.multiply(L.dot(w),R.dot(w))
assert result.all(), "result contains an inequality"

1 97 5 10 25 125 100 1250


Step 2. R1CS over Finite Field

In [104]:
import galois
GF = galois.GF(113)
x = GF(5)#random.randint(1,10)
y = GF(10)#random.randint(1,15)

v1 = x * x
v2 = v1 * x
v3 = y * y
v4 = v2 * y
out = v2 + GF(2) * v4 - GF(5) * x * v3 - GF(3) * y + GF(2)
print(1,out,x,y,v1,v2,v3,v4)

Lgf = GF(np.array([
    [0,0,1,0,0,0,0,0],
    [0,0,0,0,1,0,0,0],
    [0,0,0,1,0,0,0,0],
    [0,0,0,0,0,1,0,0],
    [0,0,108,0,0,0,0,0] # 113 - 5 = 108
    ]))

Rgf = GF(np.array([
    [0,0,1,0,0,0,0,0],
    [0,0,1,0,0,0,0,0],
    [0,0,0,1,0,0,0,0],
    [0,0,0,1,0,0,0,0],
    [0,0,0,0,0,0,1,0]
    ]))

Ogf = GF(np.array([
    [0,0,0,0,1,0,0,0],
    [0,0,0,0,0,1,0,0],
    [0,0,0,0,0,0,1,0],
    [0,0,0,0,0,0,0,1],
    [111,1,0,3,0,GF(112),0,GF(111)] # 113 - 2 = 111
    ]))

wgf = GF(np.array([GF(1),out,x,y,v1,v2,v3,v4]))

# assert all(np.multiply(np.dot(L,w), np.dot(R,w)) == np.dot(O,w)), "result contains an inequality"
result = Ogf.dot(wgf) == np.multiply(Lgf.dot(wgf),Rgf.dot(wgf))
assert result.all(), "result contains an inequality"


1 97 5 10 25 12 100 7


Step 3. Interpolate columns. 

In [106]:
target_poly = GF(np.array([1,2,3,4,5]))
# for each column in the matrix, compute the lagrange polynomial, resulting in a vector of polynomials
def interpolate_columns(column):
    return galois.lagrange_poly(target_poly,column)

L_vector_of_poly = np.apply_along_axis(interpolate_columns, 0, Lgf)
R_vector_of_poly = np.apply_along_axis(interpolate_columns, 0, Rgf)
O_vector_of_poly = np.apply_along_axis(interpolate_columns, 0, Ogf)
print(L_vector_of_poly)

[Poly(0, GF(113)) Poly(0, GF(113))
 Poly(94x^4 + 58x^3 + 71x^2 + 4x, GF(113))
 Poly(85x^4 + 110x^3 + 97x^2 + 37x + 10, GF(113))
 Poly(94x^4 + 21x^3 + 9x^2 + 112x + 103, GF(113))
 Poly(94x^4 + 96x^3 + 12x^2 + 29x + 108, GF(113)) Poly(0, GF(113))
 Poly(0, GF(113))]


Step 4. Reduce m polynomials to 1 by Inner Product of Vector of Polynomials with Witness Vector

In [120]:
def reduce_polynomials_to_1(vector_of_poly, witness):
    sum = GF(0)
    for i in range(len(vector_of_poly)):
        sum += vector_of_poly[i]*witness[i]
    return sum

Ux = reduce_polynomials_to_1(L_vector_of_poly,wgf)
Vx = reduce_polynomials_to_1(R_vector_of_poly,wgf)
Wx = reduce_polynomials_to_1(O_vector_of_poly,wgf)
UxCoeffs = Ux.coeffs
VxCoeffs = Vx.coeffs
WxCoeffs = Wx.coeffs
print(UxCoeffs)
print(VxCoeffs)
print(WxCoeffs)

from numpy import poly1d
u = poly1d([ 52,  16, 112,  35,  16])
v = poly1d([75, 77, 95, 85, 12])
w = poly1d([27, 22, 91, 59, 52])
print(u *v)

t = poly1d([1, -1])*poly1d([1, -2])*poly1d([1, -3])*poly1d([1, -4])
print(t)
print((u*v- w) // t)

[ 52  16 112  35  16]
[75 77 95 85 12]
[27 22 91 59 52]
      8        7             6             5             4
3900 x + 5204 x + 1.457e+04 x + 1.719e+04 x + 1.652e+04 x
              3        2
 + 1.427e+04 x + 5839 x + 1780 x + 192
   4      3      2
1 x - 10 x + 35 x - 50 x + 24


TypeError: unsupported operand type(s) for //: 'poly1d' and 'poly1d'

We now have 3 polynomials, U(x), V(x), W(x), i.e. U(x) = sum of L's polynomials inner product with the witness.  
U(x) is a direct representation of the Matrix Vector combination from R1CS.  
This is our Quadratic Arithmetic Program.

Notation in G16 paper shows  
$(U \cdot a)$  as     $\sum_{i=0}^m a_iu_i(X)$
- $m$ is the height of the matrix and also length of the witness
- $u_i$ is the polynomial output from lagrange interpolation $<u_1,u_2,..>$
- $a_i$ is the witness $<a_1,a_2,..>$

$(U \cdot a)\odot(V \cdot a) = (W \cdot a)$

$(U \cdot a) =  52x^4 + 16x^3 + 112x^2 + 35x + 16$ is a degree 4 polynomial  
$(V \cdot a) =  75x^4 + 77x^3 + 95x^2 + 85x + 12$ is a degree 4 polynomial  
$(W \cdot a) =  27x^4 + 22x^3 + 91x^2 + 59x + 52$ is a degree 4 polynomial  
Therefore, need to fix W to accomodate the max degree (U * V = 4 + 4 = degree 8)

$(U \cdot a)\odot(V \cdot a) = (W \cdot a)$  = unbalanced  
$(U \cdot a)\odot(V \cdot a) = (W \cdot a) \square (something) $ == balanced  
solution  
$(U \cdot a)\odot(V \cdot a) = (W \cdot a) + 0$

What is something that can balance it?
A polynomial that interpolates $y=0$ at each x. 

$(x-1)(x-2)(x-3)(x-4)$ 

$target \, polynomial =$ $24x^4 -50x^3  + 35x^2 -10x + 1$

In [92]:
print(np.polynomial.polynomial.polyfromroots([1,2,3,4]))
# 24x^4 -50x^3  + 35x^2 -10x + 1

[ 24. -50.  35. -10.   1.]


Is the target polynomial enough to balance it?  (U·a)(V·a) = (W·a) + t(x)

Step 5. Balancing the QAP with $h(X)t(X)$ i.e. $(U \cdot a)\odot(V \cdot a) = (W \cdot a) + h(X)t(X)$

In [135]:
# (UxVx - Wx) / Tx = Hx
UxVx = Ux*Vx
Tx = galois.Poly(np.flip(np.polynomial.polynomial.polyfromroots([1,2,3,4])),  field=GF)
Hx = ((Ux*Vx) - Wx) // Tx 
print("Tx:",Tx)
print("Hx: ", Hx)
print("UxVx    : ",UxVx)
print("Wx + Hx*Tx",((Hx*Tx) + Wx))

Tx: x^4 + 103x^3 + 35x^2 + 63x + 24
Hx:  58x^4 + 21x^3 + 96x^2 + 87x + 100
UxVx    :  58x^8 + 6x^7 + 108x^6 + 13x^5 + 21x^4 + 31x^3 + 76x^2 + 85x + 79
Wx + Hx*Tx 58x^8 + 6x^7 + 108x^6 + 13x^5 + 21x^4 + 31x^3 + 76x^2 + 85x + 79


Step 6. Schwartz-Zippel Lemma
2 polynomials are likely to be the same if they evaluate to the same result given a random x

In [93]:
poly_1 = UxVx
poly_2 = Wx + (Hx * Tx)
print(np.polyval(poly_1.coeffs,5))
print(np.polyval(poly_2.coeffs,5))

# print(np.polyval(poly_1,5))
# np.polyval(poly_2,5)
# print(poly_1)
# print(poly_2)

24872529
24870256


In [128]:
field_size = 113

def multiply_polynomials(poly1, poly2):
    result = [0] * (len(poly1) + len(poly2) - 1)
    
    for i in range(len(poly1)):
        for j in range(len(poly2)):
            result[i + j] += poly1[i] * poly2[j]
            result[i + j] %= field_size
    
    return result

poly_x_minus_1 = [1, -1]
poly_x_minus_2 = [1, -2]
poly_x_minus_3 = [1, -3]
poly_x_minus_4 = [1, -4]

result_poly = [1]  # Initialize with 1 as the multiplicative identity in a field

result_poly = multiply_polynomials(result_poly, poly_x_minus_1)
result_poly = multiply_polynomials(result_poly, poly_x_minus_2)
result_poly = multiply_polynomials(result_poly, poly_x_minus_3)
result_poly = multiply_polynomials(result_poly, poly_x_minus_4)

print(result_poly)
print(np.polynomial.polynomial.polyfromroots(result_poly))

[1, 103, 35, 63, 24]
[-5.450760e+06  5.973051e+06 -5.394140e+05  1.734800e+04 -2.260000e+02
  1.000000e+00]
